In [5]:
import cv2
import mediapipe as mp
import numpy as np
import open3d as o3d

In [6]:
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(
    static_image_mode=False,
    model_complexity=1,
    min_detection_confidence=0.5
)

In [7]:
vis_3d = o3d.visualization.Visualizer()
vis_3d.create_window(window_name="3D_Visual", width = 900, height = 480)

point_cloud = o3d.geometry.PointCloud()
line_set = o3d.geometry.LineSet()
vis_3d.add_geometry(point_cloud)
vis_3d.add_geometry(line_set)

ground_plane = o3d.geometry.TriangleMesh.create_box(width=4.0, height=0.02, depth=4.0)
ground_plane.translate((-2.0, -1.0, -2.0))
ground_plane.paint_uniform_color([0.7, 0.7, 0.7])
vis_3d.add_geometry(ground_plane)

connections = mp_pose.POSE_CONNECTIONS

video_path = "bowling.mp4"
cap = cv2.VideoCapture(video_path)

while cap.isOpened():
    success, image = cap.read()
    if not success:
        print("Video ended. Looping...")
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
        continue

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    if results.pose_landmarks:
        mp_drawing.draw_landmarks(
            image, 
            results.pose_landmarks, 
            connections,
            landmark_drawing_spec=mp_drawing.DrawingSpec(color=(255, 0, 255), thickness=2, circle_radius=2), # Magenta joints
            connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 255), thickness=2, circle_radius=2)  # Cyan bones
        )

    

    if results.pose_world_landmarks:
        landmarks_3d = results.pose_world_landmarks.landmark

        #angle
        
        r_shoulder = landmarks_3d[mp_pose.PoseLandmark.RIGHT_SHOULDER]
        r_elbow = landmarks_3d[mp_pose.PoseLandmark.RIGHT_ELBOW]
        r_wrist = landmarks_3d[mp_pose.PoseLandmark.RIGHT_WRIST]

        l_shoulder = landmarks_3d[mp_pose.PoseLandmark.LEFT_SHOULDER]
        l_elbow = landmarks_3d[mp_pose.PoseLandmark.LEFT_ELBOW]
        l_wrist = landmarks_3d[mp_pose.PoseLandmark.LEFT_WRIST]

        s_r = np.array([r_shoulder.x, r_shoulder.y, r_shoulder.z])
        e_r = np.array([r_elbow.x, r_elbow.y, r_elbow.z])
        w_r = np.array([r_wrist.x, r_wrist.y, r_wrist.z])

        s_l = np.array([l_shoulder.x, l_shoulder.y, l_shoulder.z])
        e_l = np.array([l_elbow.x, l_elbow.y, l_elbow.z])
        w_l = np.array([l_wrist.x, l_wrist.y, l_wrist.z])

        def angle_calc(a,b,c):
            ba = a - b
            bc = c - b
            cos_theta = np.dot(ba,bc)/(np.linalg.norm(ba)*np.linalg.norm(bc))
            angle = np.arccos(np.clip(cos_theta, -1.0, 1.0))
            return np.degrees(angle)

        r_elbow_angle = angle_calc(s_r,e_r,w_r)
        l_elbow_angle = angle_calc(s_l,e_l,w_l)

        landmarks_2d = results.pose_landmarks.landmark
        image_h, image_w, _ = image.shape
    
    # Get pixel coordinates for the elbows
        right_elbow_2d = landmarks_2d[mp_pose.PoseLandmark.RIGHT_ELBOW]
        left_elbow_2d = landmarks_2d[mp_pose.PoseLandmark.LEFT_ELBOW]
        coordinates_elbow_r = (int(right_elbow_2d.x * image_w), int(right_elbow_2d.y * image_h))
        coordinates_elbow_l = (int(left_elbow_2d.x * image_w), int(left_elbow_2d.y * image_h))

        cv2.putText(image, str(int(r_elbow_angle)),tuple(coordinates_elbow_r), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)
        cv2.putText(image, str(int(l_elbow_angle)),tuple(coordinates_elbow_l), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)
            
        
        #angle

        points_3d = np.array([[lm.x, -lm.y, -lm.z] for lm in landmarks_3d])
        visibilities = np.array([lm.visibility for lm in landmarks_3d])

        visible_points = points_3d[visibilities > 0.1]
        point_cloud.points = o3d.utility.Vector3dVector(visible_points)
        point_cloud.paint_uniform_color([1.0, 0.0, 1.0])

        lines = []
        for connection in connections:
            a, b = connection
            if visibilities[a] > 0.1 and visibilities[b] > 0.1:
                lines.append([a, b])
        
        line_set.points = o3d.utility.Vector3dVector(points_3d)
        line_set.lines = o3d.utility.Vector2iVector(lines)
        line_set.paint_uniform_color([0.0, 1.0, 1.0])

        vis_3d.update_geometry(point_cloud)
        vis_3d.update_geometry(line_set)

    resized_img = cv2.resize(image, (1920,1080))
    cv2.imshow('2D Pose Overlay', resized_img)

    if not vis_3d.poll_events():
        break
    vis_3d.update_renderer()

    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
vis_3d.destroy_window()
pose.close()

[Open3D WARNING] The number of points is 0 when creating axis-aligned bounding box.
[Open3D WARNING] The number of points is 0 when creating axis-aligned bounding box.
